# L4b: Breadth-First Search (BFS) and Depth-First Search (DFS) on a Graph
This lab will familiarize students with [Breadth-First Search](CHEME-5800-L4a-Algorithm-BFS-Fall-2025.ipynb) and [Depth-First Search](CHEME-5800-L4a-Algorithm-DFS-Fall-2025.ipynb) graph traversal on a simple graph example. 

> __Learning Objectives__
> 
> * **Build and validate directed graph models from edge list data.** You'll learn how to construct graph data structures from file-based edge lists and use unit testing to verify that your graph implementation is mathematically sound and structurally correct.
> * **Implement and compare DFS and BFS traversal algorithms.** We'll explore how these fundamental algorithms work differently on the same graph structure, particularly seeing how directed edges constrain which nodes you can actually reach from different starting points.
> * **Apply graph theory concepts through hands-on computation.** You'll get practical experience with concepts like the Handshaking Lemma and degree calculations, connecting theoretical graph properties to real algorithmic implementations.

The practical implementation demonstrates how these fundamental traversal algorithms behave differently on the same graph structure, particularly highlighting how directed edges constrain reachability from different starting nodes.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our environment:

In [1]:
include(joinpath(@__DIR__, "Include-student.jl")); # what is this doing?

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

___

<div>
    <center>
        <img src="figs/Fig-Example-Graph.svg" width="480"/>
    </center>
</div>

## Task 1: Build a graph model instance for an example graph using an edge list
In this task, we'll build [a `MySimpleDirectedGraphModel` instance](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MySimpleDirectedGraphModel) from an edge list file stored in the `data` folder. This edge list encodes the graph shown above.


### Aside: Adjacency Matrix versus Edge List
A graph $\mathcal{G}=\left(\mathcal{V},\mathcal{E}\right)$ can be constructed from an [Adjacency Matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) $\mathcal{A}$, which is a $|\mathcal{V}|\times|\mathcal{V}|$ matrix. However, this is only suitable for _small graphs_ because $\mathcal{A}$ has a high memory overhead (if stored as `64-bit` values).

For example, consider a graph with $|\mathcal{V}|$ = 100,000 nodes which would require `80 GB` of memory to store in the worst case, which is more memory than most common machines have.

In [2]:
𝒱 = 100000;
memory_reqd = (𝒱^2)*8*(1/(1e9)) # 8 x bytes (64-bits) for each entry = units GB

80.0

Instead, a lower memory representation is an [Edge list representation](https://en.wikipedia.org/wiki/Edge_list). In the [Edge list representation](https://en.wikipedia.org/wiki/Edge_list), only the edge information is stored (typically) in a comma-separated value (CSV) file in which each record holds an edge in the graph:

> __Edge list__: Each record in an edge list file will contain at least the source, target, and weight fields (for a weighted graph). There can be variations in the exact format, but this is the general structure. For example, there could be id, source, target, weight. The delimiter is typically a comma, but could also be a tab or space, etc.

We've built [the `MyGraphEdgeModels(...)` function](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/data/#VLDataScienceMachineLearningPackage.MyGraphEdgeModels) which takes a file path as an argument, and returns a dictionary of [`MyGraphEdgeModel` instances](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MyGraphEdgeModel) to hold this information. Let's load an edge list. 

First, set the path to the edge list file in the `path_to_edge_file::String` variable:

In [3]:
path_to_edge_file = joinpath(_PATH_TO_DATA, "SimpleGraph.txt"); # this will load the graph shown above

Next, construct our dictionary of `edgemodels`, where the data for the edges (source id, target id, and weight) is stored in [a `MyGraphEdgeModel` instance](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MyGraphEdgeModel).

Because each edge file could have a different record structure, let's define a __callback function__ that will be called for each line of the edge file. This function will parse the record in the edge file, and return the source id, target id, and weight as a tuple. Note that the internal data structure uses the `Any` data type for the weight, so this can be a string, float, integer, or an object, etc.

In this case, we'll keep it simple: we'll just return the weight as a float.

In [4]:
function edgerecordparser(record::String, delim::Char=',')
    
    fields = split(record, delim) # this assumes a record of the form "source,target,weight"
    if length(fields) < 3
        return nothing
    end

    source = parse(Int, fields[1]) # source id
    target = parse(Int, fields[2]) # target id
    weight = parse(Float64, fields[3]) # edge weight
    
    return (source, target, weight)
end

edgerecordparser (generic function with 2 methods)

Now that we have our path to the edge list file, and the callback function `edgerecordparser` defined, we can create our edgemodel dictionary by calling [the `MyGraphEdgeModels(...)` function](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/data/#VLDataScienceMachineLearningPackage.MyGraphEdgeModels).

Let's save our edge models in the `myedgemodels::Dict{Int64, MyGraphEdgeModel}` dictionary. The keys will be the edge ids (which we can assume are unique), and the values will be the corresponding `MyGraphEdgeModel` instances. Here, we've used the line index as the edge id.

In [5]:
myedgemodels = MyGraphEdgeModels(path_to_edge_file, edgerecordparser, delim=',', comment='#')

Dict{Int64, MyGraphEdgeModel} with 7 entries:
  0 => MyGraphEdgeModel(0, 1, 2, 10.0)
  4 => MyGraphEdgeModel(4, 3, 5, 6.0)
  5 => MyGraphEdgeModel(5, 4, 6, 1.0)
  6 => MyGraphEdgeModel(6, 5, 4, 1.0)
  2 => MyGraphEdgeModel(2, 2, 3, 2.0)
  3 => MyGraphEdgeModel(3, 2, 4, 100.0)
  1 => MyGraphEdgeModel(1, 1, 3, 100.0)

How can we tell what are the fields in [the `MyGraphEdgeModel` type](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MyGraphEdgeModel)?

In [6]:
typeof(myedgemodels[1]) |> m-> fieldnames(m)

(:id, :source, :target, :weight)

Finally, we can build a graph instance. Since this is a directed graph, we'll construct [a `MySimpleDirectedGraphModel` instance](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MySimpleDirectedGraphModel) using [a `build(...)` method](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/factory/#VLDataScienceMachineLearningPackage.build). Let's save our graph model in the `directedgraphmodel::MySimpleDirectedGraphModel` variable.

In [7]:
directedgraphmodel = build(MySimpleDirectedGraphModel, myedgemodels)

MySimpleDirectedGraphModel(Dict{Int64, MyGraphNodeModel}(5 => MyGraphNodeModel(5, nothing), 4 => MyGraphNodeModel(4, nothing), 6 => MyGraphNodeModel(6, nothing), 2 => MyGraphNodeModel(2, nothing), 3 => MyGraphNodeModel(3, nothing), 1 => MyGraphNodeModel(1, nothing)), Dict{Tuple{Int64, Int64}, Number}((2, 4) => 100.0, (1, 2) => 10.0, (1, 3) => 100.0, (4, 6) => 1.0, (3, 5) => 6.0, (5, 4) => 1.0, (2, 3) => 2.0), Dict{Int64, Set{Int64}}(5 => Set([4]), 4 => Set([6]), 6 => Set(), 2 => Set([4, 3]), 3 => Set([5]), 1 => Set([2, 3])), Dict(5 => (3, 5), 4 => (2, 4), 6 => (4, 6), 7 => (5, 4), 2 => (1, 3), 3 => (2, 3), 1 => (1, 2)))

What fields are in [the `MySimpleDirectedGraphModel` type](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/types/#VLDataScienceMachineLearningPackage.MySimpleDirectedGraphModel)?

In [8]:
typeof(directedgraphmodel) |> m-> fieldnames(m)

(:nodes, :edges, :children, :edgesinverse)

In [9]:
directedgraphmodel.nodes

Dict{Int64, MyGraphNodeModel} with 6 entries:
  5 => MyGraphNodeModel(5, nothing)
  4 => MyGraphNodeModel(4, nothing)
  6 => MyGraphNodeModel(6, nothing)
  2 => MyGraphNodeModel(2, nothing)
  3 => MyGraphNodeModel(3, nothing)
  1 => MyGraphNodeModel(1, nothing)

### Check: Is our graph structure correct?
We've built a simple test graph, with a known format, so we can do some unit tests to determine if various theoretical and implementation aspects about the graph, e.g., the connectivity, the number of edges, nodes, etc., are correct.

Let's use [the `@testset` macro](https://docs.julialang.org/en/v1.6/stdlib/Test/#Test.@testset) to group our various tests together, where each test will use [the `@test` macro](https://docs.julialang.org/en/v1.6/stdlib/Test/#Test.@test) to assert the expected behavior.

Our test suite validates several key aspects of the graph construction and mathematical properties. First, we verify that the data structures are correctly instantiated as the expected types and that the graph has the anticipated size matching the visual diagram (6 nodes and 7 edges). We also check that the node identifiers are consecutive integers starting from 1, which is a common convention in graph representations.

* __Test Consistency:__ The tests ensure consistency between different internal representations of the same graph data. We verify that the edge information stored in the `directedgraphmodel.edges` dictionary matches exactly with the edge data in our `myedgemodels` collection.
* __Structural Integrity:__ We validate that all edge endpoints reference valid nodes that actually exist in the graph. We also test the consistency of the `children` field, which should contain exactly the set of target nodes for each source node based on the directed edges.
* __Mathematical Validity:__ We apply basic graph-theoretic bounds to ensure the structure is mathematically valid. For a directed graph with $n$ nodes, we verify that our edge count $m$ satisfies $m \leq n(n-1)$ and that the graph is non-empty with $m > 0$.

Do we pass these tests?

In [10]:
let
   
    @testset verbose = true "Graph Structure Validation" begin
        
        # Test 1: Basic types and construction
        @test isa(directedgraphmodel, MySimpleDirectedGraphModel)
        @test isa(myedgemodels, Dict{Int64, MyGraphEdgeModel})
        
        # Test 2: Expected graph size (from visual inspection of the figure)
        @test length(directedgraphmodel.nodes) == 6
        @test length(myedgemodels) == 7
        
        # Test 3: Node IDs are consecutive integers starting from 1
        node_ids = sort(collect(keys(directedgraphmodel.nodes)))
        @test node_ids == [1, 2, 3, 4, 5, 6]
        
        # Test 4: Edge consistency - edges in directedgraphmodel.edges should match myedgemodels
        edge_tuples_from_graph = Set(keys(directedgraphmodel.edges))
        edge_tuples_from_models = Set((edge.source, edge.target) for edge in values(myedgemodels))
        @test edge_tuples_from_graph == edge_tuples_from_models
        
        # Test 5: All edge endpoints exist as nodes
        for edge in values(myedgemodels)
            @test edge.source in keys(directedgraphmodel.nodes)
            @test edge.target in keys(directedgraphmodel.nodes)
        end
        
        # Test 6: Children field consistency with edges
        for (node_id, children_set) in directedgraphmodel.children
            # Children of node_id should be all targets where node_id is source
            expected_children = Set(edge.target for edge in values(myedgemodels) if edge.source == node_id)
            @test children_set == expected_children
        end
        
        # Test 7: Basic graph property bounds
        n = length(directedgraphmodel.nodes)
        m = length(myedgemodels)
        @test m ≤ n * (n - 1)  # Can't have more edges than maximum possible for directed graph
        @test m > 0  # Should have at least one edge
        
        println("✅ All graph structure validation tests passed!")
    end;
end;

✅ All graph structure validation tests passed!
Test Summary:              | Pass  Total  Time
Graph Structure Validation |   28     28  0.4s


### The Handshaking Lemma and Directed Graphs
Let's check the Handshaking Lemma for our directed graph example.

> __Handshaking Lemma__ For any graph $\mathcal{G} = (\mathcal{V},\mathcal{E})$, the sum of all vertex degrees equals twice the number of edges:
> $$\sum_{v_{i} \in \mathcal{V}} \deg(v_{i}) = 2|\mathcal{E}|$$
> This relationship holds because each edge contributes exactly one to the degree of each of its two endpoints.

The code block below calculates the total degree for each vertex in our directed graph, where total degree = in-degree + out-degree. Here's what each part does:

1. **Initialize variables**: We get the number of edges and nodes from our graph model, then allocate an array to store the total degree of each node.
2. **Calculate total degrees**: The nested loops iterate through all possible node pairs $(i,j)$ and check if there's a directed edge from $i$ to $j$. For each existing edge $(i,j)$, we increment the degree count for both vertex $i$ (contributing to its out-degree) and vertex $j$ (contributing to its in-degree).
3. **Return results**: The block returns both the degree array (containing the total degree of each node) and the total number of edges for validation.

We store the degree data in the `degree_array::Array{Int64,1}` variable, and the number of edges in the `number_of_edges::Int64` variable.

In [16]:
degree_array, number_of_edges = let

    # initialize -
    number_of_edges = directedgraphmodel.edges |> length; # how many edges do we have?
    number_of_nodes = directedgraphmodel.nodes |> length; # how many nodes do we have?
    degree_array = Array{Int64,1}(undef, number_of_nodes); # allocate some space for the degrees

    # fill the degree_array with zeros -
    fill!(degree_array,0)

    # compute the degree -
    for i ∈ 1:number_of_nodes
        for j ∈ 1:number_of_nodes
            test_edge_coordinates = (i,j) # tuple
            if (haskey(directedgraphmodel.edges, test_edge_coordinates) == true)
                degree_array[i] += 1;
                degree_array[j] += 1;
            end
        end
    end

    degree_array, number_of_edges;
end

([2, 3, 3, 3, 2, 1], 7)

Let's test our degree calculation.

> __Test__: Let's use [the `@assert` macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to verify that our degree calculation satisfies the directed graph version of the degree sum property from the Handshaking Lemma. If this condition is `false`, the program will throw [an `AssertionError`](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) and halt execution. Otherwise, if nothing happens, we can assume the test passed.

So, what do we see?

In [17]:
@assert sum(degree_array) == 2*number_of_edges

___

## Task 2: Depth-First Search (DFS)
In this task, you will implement the Depth-First Search (DFS) algorithm to explore a graph. DFS is a fundamental recursive algorithm used for traversing or searching tree or graph data structures. The algorithm starts at a selected node and explores as far as possible along each branch before backtracking.

We've implemented basic depth-first search traversal logic [in the `mywalk(...)` function](src/Graphs.jl). This function takes a graph (or tree model) and a starting node as input and returns an ordered list of all nodes visited during the traversal. There are two additional (optional) arguments:
* __algorithm__: The `algorithm::AbstractGraphTraversalAlgorithm` argument allows you to specify which traversal algorithm to use (e.g., `DepthFirstSearchAlgorithm` or `BreadthFirstSearchAlgorithm`). If not provided, the `BreadthFirstSearchAlgorithm` algorithm will be used by default.
* __verbose__: The `verbose::Bool` argument allows you to enable or disable verbose output during the traversal. If set to `true`, the function will print detailed information about the traversal process.

Let's do a depth-first traversal of the `directedgraphmodel::MySimpleDirectedGraphModel` starting from node `1`. We'll save the traversal order in a variable called `dfs_order::Array{Int64,1}`.

In [11]:
dfs_order = let
    
    # initialize -
    verbose = true; # let's print some output
    startnode = directedgraphmodel.nodes[1]; # start at node 1
    order = mywalk(directedgraphmodel, startnode, 
        algorithm = DepthFirstSearchAlgorithm(), verbose = verbose) # walk from startnode to all other possible nodes

    order;
end;

Visiting node: 1
Visiting node: 2
Visiting node: 4
Visiting node: 6
Visiting node: 3
Visiting node: 5


### Hmm. This is a directed graph, can we get to every node?
__No!__ In a directed graph, edges have a direction, meaning you can only traverse from one node to another if there is a directed edge pointing to the destination node. This can result in some nodes being unreachable from others, depending on the structure of the graph.

Let's show this by selecting a different starting node and observing the traversal order, e.g., `startnode = 3`.

In [12]:
let
    
    # initialize -
    verbose = true; # let's print some output
    startnode = directedgraphmodel.nodes[3]; # start at node 3
    _ = mywalk(directedgraphmodel, startnode, 
        algorithm = DepthFirstSearchAlgorithm(), verbose = verbose) # walk from startnode to all other possible nodes
end;

Visiting node: 3
Visiting node: 5
Visiting node: 4
Visiting node: 6


__No!__ When we start at node `3` in our test graph, we can see that the traversal is limited to the nodes that are reachable from `3`. This highlights the nature of directed graphs, where the direction of edges determines the accessibility of nodes.

___

## Task 3: Breadth-First Search (BFS)
In this task, we'll compute the breadth-first traversal order of the directed graph starting from a specified node. We'll use the `BreadthFirstSearchAlgorithm` to explore the graph level by level.

We'll start by selecting a node to begin our search. For this example, let's choose node `1` (the same as our original DFS search). We'll save the traversal order in a variable called `bfs_order::Array{Int64,1}`.

In [13]:
bfs_order = let
    
    # initialize -
    verbose = true; # let's print some output
    startnode = directedgraphmodel.nodes[1]; # start at node 1
    order = mywalk(directedgraphmodel, startnode, 
        algorithm = BreadthFirstSearchAlgorithm(), verbose = verbose) # walk from startnode to all other nodes

    order;
end;

Visiting node: 1. My Queue: Queue{Int64}(Deque [Int64[]])
Visiting node: 2. My Queue: Queue{Int64}(Deque [[3]])
Visiting node: 3. My Queue: Queue{Int64}(Deque [[4, 3]])
Visiting node: 4. My Queue: Queue{Int64}(Deque [[3, 5]])
Visiting node: 5. My Queue: Queue{Int64}(Deque [[6]])
Visiting node: 6. My Queue: Queue{Int64}(Deque [Int64[]])


__Interesting!__ The order is different than the DFS traversal order. This is expected, as BFS explores all neighbors at the present depth prior to moving on to nodes at the next depth level, while DFS explores as far as possible along each branch before backtracking.

Okay, great! Can we get to every node in the graph using BFS?

In [14]:
let
    
    # initialize -
    verbose = true; # let's print some output
    startnode = directedgraphmodel.nodes[3]; # start at node 3
    _ = mywalk(directedgraphmodel, startnode, 
        algorithm = DepthFirstSearchAlgorithm(), verbose = verbose) # walk from startnode to all other possible nodes
end;

Visiting node: 3
Visiting node: 5
Visiting node: 4
Visiting node: 6


___

## Summary
In this notebook, we explored the concepts of Breadth-First Search (BFS) and Depth-First Search (DFS) algorithms on a directed graph. We implemented both algorithms to traverse the graph and compared their behavior and output.

`Unhide` the code block below to see how to make a table that summarizes the paths taken by both algorithms.
> __Summary:__ For a general graph, the paths taken by depth-first and breadth-first traversal will __not__ be equal. However, given the same starting node and graph structure, the two algorithms may produce the same path in certain cases, particularly in simple or linear graphs. Thus, both approaches will also always visit the same set of reachable nodes, ensuring that all reachable nodes are explored, albeit in a different order.

__So why do we care about the differences between DFS and BFS?__ These traversal algorithms are foundational in the sense that we build things from them. They have different use cases and performance characteristics. DFS is often more memory-efficient, while BFS is better suited for finding the shortest path in unweighted graphs, as it explores all neighbors at the present depth prior to moving on to nodes at the next depth level.

__DFS versus BFS__? Choose DFS for deep structure exploration (cycle detection, topological sorting) or when memory is limited in tree-like graphs, as it uses $O(h)$ space versus BFS's $O(b)$ space where $h$ is depth and $b$ is maximum nodes per level. Choose BFS when you need shortest paths in unweighted graphs or when targets are likely close to the starting node.

In [15]:
let

    # initialize -
    df = DataFrame();
    number_of_nodes = length(directedgraphmodel.nodes);

    for i ∈ 2:number_of_nodes
        
        # dfs -
        s = dfs_order[i-1];
        t = dfs_order[i];
        dfs_path = "node $(s) ->  node $(t)"
        
        # bfs -
        s = bfs_order[i-1];
        t = bfs_order[i];
        bfs_path = "node $(s) ->  node $(t)"

        row_data = (
            path = i - 1,
            dfspath = dfs_path,
            bfspath = bfs_path,
            equal = (dfs_path == bfs_path) ? true : false
        );
        
        push!(df, row_data);
    end

    # make a table -
    pretty_table(df, backend = :text, 
        table_format = TextTableFormat(borders = text_table_borders__simple))

end

======== =================== =================== ========
   path             dfspath             bfspath   equal 
  Int64              String              String    Bool 
======== =================== =================== ========
      1   node 1 ->  node 2   node 1 ->  node 2    true
      2   node 2 ->  node 4   node 2 ->  node 3   false
      3   node 4 ->  node 6   node 3 ->  node 4   false
      4   node 6 ->  node 3   node 4 ->  node 5   false
      5   node 3 ->  node 5   node 5 ->  node 6   false
======== =================== =================== ========
